In [ ]:
# SERVER

import socket
import random
import re
import pickle
import cloudpickle
from collections import Counter


host = '127.0.0.1'           # Canalul de comunicare / localhost
port = 8881                  # Portul pe care se asculta

N1 = ""
N2 = ""
N3 = ""    
N4 = ""         
N5 = ""         
N6 = ""
BONUS = ""
JOKER = ""
TRIPLA = ""
CHINTA = ""
FULL = ""
CAREU = ""
YAMS = ""
TOTAL = ""

bonus = 0

R = 2
zaruri = []
pastrez = []

                  
def start():
    # Se face legatura intre CLIENT si SERVER.
    serverSocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    serverSocket.bind((host, port))
    serverSocket.listen(2)
    print("Serverul asculta pe adresa: ", serverSocket.getsockname())
    
    variabileGlobale = globals()

    while True:
        # Se accepta solicitarea de conectare a unui CLIENT TCP.
        connectionSocket, clientAddress = serverSocket.accept()
        print("Accesat de catre: ", clientAddress)
        
        # Serverul primeste un mesaj de la client.
        stareJocClient = connectionSocket.recv(4096)

        # La fiecare joc nou, variabilele globale ale punctajelor sunt resetate.
        variabileGlobale['N1'] = ""
        variabileGlobale['N2'] = ""
        variabileGlobale['N3'] = ""   
        variabileGlobale['N4'] = ""        
        variabileGlobale['N5'] = ""         
        variabileGlobale['N6'] = ""
        variabileGlobale['BONUS'] = ""
        variabileGlobale['JOKER'] = ""
        variabileGlobale['TRIPLA'] = ""
        variabileGlobale['CHINTA'] = ""
        variabileGlobale['FULL'] = ""
        variabileGlobale['CAREU'] = ""
        variabileGlobale['YAMS'] = ""
        variabileGlobale['TOTAL'] = ""
        variabileGlobale['bonus'] = 0

        # Variabila ce ne aduce la cunostinta numarul de relansari posibile.
        R = 2
        # Variabila ce ne spune cand jocul e gata.
        stopJoc = 11

        # In functie de mesajul primit de la client, serverul decide daca comanda este una corecta sau nu.
        # In cazul in care este CORECTA, serverul ii raspunde clientului cu stareJoc si functia punctaj.
        # In cazul in care NU ESTE CORECTA, serverul ii raspunde clientului cu un mesaj ce presupune o eroare.
        if stareJocClient.decode() == "START":
            stareJoc = "start"
            connectionSocket.send(stareJoc.encode())

            functieCatreClient = cloudpickle.dumps(PUNCTAJ)
            connectionSocket.send(functieCatreClient)

            # Variabile folosite pentru a iesi din while-uri, atunci cand este cazul.
            stareJocClient2 = stareJocClient
            stareJocClient3 = stareJocClient

            # Daca totul este in regula pana aici, atunci serverul poate primi noi comenzi de la client.
            while stareJocClient2.decode() == "START":                 
                comandaClient = connectionSocket.recv(4096)

                stareJocClient3 = stareJocClient2

                # Comanda "ARUNCA", genereaza o lista de 5 zaruri si o trimite clientului.
                # Totodata, este trimis si numarul de relansari posibile.
                if comandaClient.decode() == "ARUNCA":
                    comanda = "arunca"
                    connectionSocket.send(comanda.encode())

                    R = 2

                    ARUNCA()
                    zaruriCatreClient = pickle.dumps(variabileGlobale['zaruri'])
                    connectionSocket.send(zaruriCatreClient)

                    connectionSocket.send(str(R).encode())

                    while stareJocClient3.decode() == "START":
                        comandaClient2 = connectionSocket.recv(4096)

                        # Stringul obtinut de la client este impartit
                        fct = comandaClient2.decode()
                        functie = ""
                        functie = fct[fct.find('""')+1:fct.find('(')]

                        variabileGlobale = globals()
                        zaruri = variabileGlobale['zaruri']

                        # De aici, daca clientul doreste adaugarea punctajului in tabel, serverul va actualiza tabelul si il va prezenta clientului.
                        if (comandaClient2.decode() == "N1" and N1 == "") or (comandaClient2.decode() == "N2" and N2 == "") or (comandaClient2.decode() == "N3" and N3 == "") or (comandaClient2.decode() == "N4" and N4 == "") or (comandaClient2.decode() == "N5" and N5 == "") or (comandaClient2.decode() == "N6" and N6 == "") or (comandaClient2.decode() == "JOKER" and JOKER == "") or (comandaClient2.decode() == "TRIPLA" and TRIPLA == "") or (comandaClient2.decode() == "CHINTA" and CHINTA == "") or (comandaClient2.decode() == "FULL" and FULL == "") or (comandaClient2.decode() == "CAREU" and CAREU == "") or (comandaClient2.decode() == "YAMS" and YAMS == ""):

                            # Daca stopJoc este diferit de 0, jocul va continua.
                            # Daca stopJoc atinge valoarea 0, jocul va fi considerat terminat. Serverul va fi pregatit apoi pentru o noua sesiune de YAMS.
                            if  stopJoc != 0:
                                comanda2 = comandaClient2.decode()
                                connectionSocket.send(comanda2.encode())

                                comandaJocTerminat = "continua"
                                connectionSocket.send(comandaJocTerminat.encode())

                                adaugaInTabel(comanda2)
                                stopJoc -= 1

                                functieCatreClient = cloudpickle.dumps(PUNCTAJ)
                                connectionSocket.send(functieCatreClient)

                                stareJocClient3 = comanda2.encode() 
                            else:
                                comanda2 = comandaClient2.decode()
                                connectionSocket.send(comanda2.encode())

                                comandaJocTerminat = "JOC TERMINAT"
                                connectionSocket.send(comandaJocTerminat.encode())

                                adaugaInTabel(comanda2)

                                functieCatreClient = cloudpickle.dumps(PUNCTAJ)
                                connectionSocket.send(functieCatreClient)

                                stareJocClient3 = comanda2.encode() 
                                stareJocClient2 = comanda2.encode()
                                break

                        # Serverul ii trimite clientului functia cu punctajul obtinut pana la momentul respectiv.
                        elif comandaClient2.decode() == "PUNCTAJ":
                            comanda2 = "punctaj"
                            connectionSocket.send(comanda2.encode())

                            functieCatreClient = cloudpickle.dumps(PUNCTAJ)
                            connectionSocket.send(functieCatreClient)

                        # In cazul in care clientul cere abandonul, jocul initial este pierdut si se iese din while-uri.
                        elif comandaClient2.decode() == "ABANDON":
                            comanda2 = "abandon"
                            connectionSocket.send(comanda2.encode())
                            stareJocClient3 = comanda2.encode()
                            stareJocClient2 = comanda2.encode()
                            break

                        # Daca stringul divizat indeplineste conditia, argumentele din paranteza sunt pastrate intr-o lista(la inceput o lista de stringuri, iar apoi transformate in int)
                        elif functie == "KEEP":
                            comanda2 = "keep"
                            connectionSocket.send(comanda2.encode())

                            argumente = fct[fct.find('(')+1:fct.find(')')]
                            pastrez = list(re.split(',', argumente))

                            for i in range(0, len(pastrez)):
                                pastrez[i] = int(pastrez[i])

                            # Daca multimea elementelor din lista pastrez este mai mica decat multimea elementelor din zaruri, 
                            # atunci zarurile sunt rearuncate (cu exceptia celor pastrate).

                            # In caz contrat, nu se intampla nimic, iar clientul o sa primeasca aceiasi lista de zaruri,
                            # cu acelasi numar de relansari.
                            if set(pastrez) <= set(zaruri) and R > 0 and len(pastrez) > 0 and len(pastrez) <= 5:
                                KEEP(pastrez)
                                R -= 1

                            zaruriRearuncateCatreClient = pickle.dumps(variabileGlobale['zaruri'])
                            connectionSocket.send(zaruriRearuncateCatreClient)

                            connectionSocket.send(str(R).encode())

                        # O comanda necunoscuta de catre server este o "COMANDA ERONATA" pentru client.
                        else:
                            comanda2 = "COMANDA ERONATA"
                            connectionSocket.send(comanda2.encode())

                # Serverul ii trimite clientului functia cu punctajul obtinut pana la momentul respectiv.
                elif comandaClient.decode() == "PUNCTAJ":
                    comanda = "punctaj"
                    connectionSocket.send(comanda.encode())

                    functieCatreClient = cloudpickle.dumps(PUNCTAJ)
                    connectionSocket.send(functieCatreClient)

                # In cazul in care clientul cere abandonul, jocul initial este pierdut si se iese din while.
                elif comandaClient.decode() == "ABANDON":
                    comanda = "abandon"
                    connectionSocket.send(comanda.encode())
                    break

                # O comanda necunoscuta de catre server este o "COMANDA ERONATA" pentru client.
                else:
                    comanda = "COMANDA ERONATA"
                    connectionSocket.send(comanda.encode())

        # O comanda necunoscuta de catre server este o "COMANDA ERONATA" pentru client.
        else:
            stareJoc = "COMANDA ERONATA"
            connectionSocket.send(stareJoc.encode())
                
    connectionSocket.close()
    
# Afiseaza punctajul obtinut.
def PUNCTAJ():
    print("\n")
    print('N1 -----> ', N1) 
    print('N2 -----> ', N2) 
    print('N3 -----> ', N3)
    print('N4 -----> ', N4)
    print('N5 -----> ', N5)
    print('N6 -----> ', N6)
    print('BONUS --> ', BONUS)
    print('JOKER --> ', JOKER)
    print('TRIPLA -> ', TRIPLA)
    print('CHINTA -> ', CHINTA)
    print('FULL ---> ', FULL)
    print('CAREU --> ', CAREU)
    print('YAMS ---> ', YAMS)
    print('TOTAL --> ', TOTAL)
    print("\n")

# Genereaza o lista cu 5 "zaruri" sortate, cu valori cuprinse intre 1 si 6.
def ARUNCA():
    variabileGlobale = globals()
    zaruri = [random.randint(1, 6) for zaruri in range(5)]
    variabileGlobale['zaruri'] = sorted(zaruri)

# Zarurile pastrate (args), sunt pastrate. Celelalte sunt rearuncate si mai apoi adaugate celor pastrate, si totodata sortate
def KEEP(args):
    variabileGlobale = globals()
    zaruri = variabileGlobale['zaruri']

    pastreaza = list(args)
    reArunca = list((Counter(zaruri) - Counter(pastreaza)).elements())

    reArunca = [random.randint(1, 6) for reArunca in range (len(reArunca))]
    reArunca = sorted(reArunca)
    zaruri = sorted(pastreaza + reArunca)
    variabileGlobale['zaruri'] = zaruri

# In functie de alegere, punctajul este actualizat.
# tineMinte numara de cate ori apare o anumita valoare a unui zar.
# Daca zarurile curente sunt [1, 1, 1, 4, 4], atunci tineMinte va fi o lista de forma [3, 3, 3, 2, 2] ceea ce este echivalentul unui FULL.
# Un YAMS in tineMinte va fi de forma [5, 5, 5, 5, 5].
def adaugaInTabel(message):
    
    variabileGlobale = globals()
    tineMinte = []
    
    for x in zaruri:
        tineMinte.append(zaruri.count(x))
    
    if message == "YAMS" and variabileGlobale['YAMS'] == "":
        if variabileGlobale['TOTAL'] == "":
            variabileGlobale['TOTAL'] = 0
        if 5 in tineMinte:
            variabileGlobale['YAMS'] = 50
            variabileGlobale['TOTAL'] += 50
        else:
            variabileGlobale['YAMS'] = 0
            
        
    elif message == "FULL" and variabileGlobale['FULL'] == "":
        if variabileGlobale['TOTAL'] == "":
            variabileGlobale['TOTAL'] = 0
        if 3 in tineMinte and 2 in tineMinte:
            variabileGlobale['FULL'] = 30
            variabileGlobale['TOTAL'] += 30
        else:
            variabileGlobale['FULL'] = 0
        
    elif message == "TRIPLA" and variabileGlobale['TRIPLA'] == "":
        if variabileGlobale['TOTAL'] == "":
            variabileGlobale['TOTAL'] = 0        
        if (3 in tineMinte and 2 not in tineMinte) or (3 in tineMinte and 2 in tineMinte) or (5 in tineMinte) or (4 in tineMinte) :
            variabileGlobale['TRIPLA'] = 21
            variabileGlobale['TOTAL'] += 21
        else:
            variabileGlobale['TRIPLA'] = 0
        
    elif message == "CAREU" and variabileGlobale['CAREU'] == "": 
        if variabileGlobale['TOTAL'] == "":
            variabileGlobale['TOTAL'] = 0
        if (4 in tineMinte) or (5 in tineMinte):
            variabileGlobale['CAREU'] = 40
            variabileGlobale['TOTAL'] += 40
        else:
            variabileGlobale['CAREU'] = 0
        
    elif message == "CHINTA" and variabileGlobale['CHINTA'] == "": 
        if variabileGlobale['TOTAL'] == "":
            variabileGlobale['TOTAL'] = 0        
        if zaruri[-2] == zaruri[-1] - 1 and zaruri[-3] == zaruri[-2] - 1 and zaruri[-4] == zaruri[-3] - 1 and zaruri[-5] == zaruri[-4] - 1:
            variabileGlobale['CHINTA'] = 20
            variabileGlobale['TOTAL'] += 20
        else:
            variabileGlobale['CHINTA'] = 0
            
        
    elif message == "N1" and variabileGlobale['N1'] == "":
        if variabileGlobale['TOTAL'] == "":
            variabileGlobale['TOTAL'] = 0
        if variabileGlobale['BONUS'] == "":
            variabileGlobale['BONUS'] = 0
        if variabileGlobale['N1'] == "":
            variabileGlobale['N1'] = 0
        for i in zaruri:
            if i == 1:
                variabileGlobale['N1'] += 1
                variabileGlobale['TOTAL'] += 1
            else:
                variabileGlobale['N1'] += 0

        variabileGlobale['bonus'] += N1
        
        if variabileGlobale['bonus'] >= 63 and variabileGlobale['BONUS'] == 0:
            variabileGlobale['BONUS'] = 50
            variabileGlobale['TOTAL'] += 50
  
    elif message == "N2" and variabileGlobale['N2'] == "":
        if variabileGlobale['TOTAL'] == "":
            variabileGlobale['TOTAL'] = 0
        if variabileGlobale['BONUS'] == "":
            variabileGlobale['BONUS'] = 0
        if variabileGlobale['N2'] == "":
            variabileGlobale['N2'] = 0
        for i in zaruri:
            if i == 2:
                variabileGlobale['N2'] += 2
                variabileGlobale['TOTAL'] += 2
            else:
                variabileGlobale['N2'] += 0
        
        variabileGlobale['bonus'] += N2
        
        if variabileGlobale['bonus'] >= 63 and variabileGlobale['BONUS'] == 0:
            variabileGlobale['BONUS'] = 50
            variabileGlobale['TOTAL'] += 50
     
    elif message == "N3" and variabileGlobale['N3'] == "":
        if variabileGlobale['TOTAL'] == "":
            variabileGlobale['TOTAL'] = 0
        if variabileGlobale['BONUS'] == "":
            variabileGlobale['BONUS'] = 0
        if variabileGlobale['N3'] == "":
            variabileGlobale['N3'] = 0
        for i in zaruri:
            if i == 3:
                variabileGlobale['N3'] += 3
                variabileGlobale['TOTAL'] += 3
            else:
                variabileGlobale['N3'] += 0

        variabileGlobale['bonus'] += N3
        if variabileGlobale['bonus'] >= 63 and variabileGlobale['BONUS'] == 0:
            variabileGlobale['BONUS'] = 50
            variabileGlobale['TOTAL'] += 50

            
    elif message == "N4" and variabileGlobale['N4'] == "":
        if variabileGlobale['TOTAL'] == "":
            variabileGlobale['TOTAL'] = 0
        if variabileGlobale['BONUS'] == "":
            variabileGlobale['BONUS'] = 0
        if variabileGlobale['N4'] == "":
            variabileGlobale['N4'] = 0
        for i in zaruri:
            if i == 4:
                variabileGlobale['N4'] += 4
                variabileGlobale['TOTAL'] += 4
            else:
                variabileGlobale['N4'] += 0
        
        variabileGlobale['bonus'] += N4
        if variabileGlobale['bonus'] >= 63 and variabileGlobale['BONUS'] == 0:
            variabileGlobale['BONUS'] = 50
            variabileGlobale['TOTAL'] += 50
            
    elif message == "N5" and variabileGlobale['N5'] == "":
        if variabileGlobale['TOTAL'] == "":
            variabileGlobale['TOTAL'] = 0
        if variabileGlobale['BONUS'] == "":
            variabileGlobale['BONUS'] = 0
        if variabileGlobale['N5'] == "":
            variabileGlobale['N5'] = 0
        for i in zaruri:
            if i == 5:
                variabileGlobale['N5'] += 5
                variabileGlobale['TOTAL'] += 5
            else:
                variabileGlobale['N5'] += 0

        variabileGlobale['bonus'] += N5
        if variabileGlobale['bonus'] >= 63 and variabileGlobale['BONUS'] == 0:
            variabileGlobale['BONUS'] = 50
            variabileGlobale['TOTAL'] += 50

            
    elif message == "N6" and variabileGlobale['N6'] == "":
        if variabileGlobale['TOTAL'] == "":
            variabileGlobale['TOTAL'] = 0
        if variabileGlobale['BONUS'] == "":
            variabileGlobale['BONUS'] = 0
        if variabileGlobale['N6'] == "":
            variabileGlobale['N6'] = 0
        for i in zaruri:
            if i == 6:
                variabileGlobale['N6'] += 6
                variabileGlobale['TOTAL'] += 6
            else:
                variabileGlobale['N6'] += 0
        
        variabileGlobale['bonus'] += N6
        if variabileGlobale['bonus'] >= 63 and variabileGlobale['BONUS'] == 0:
            variabileGlobale['BONUS'] = 50
            variabileGlobale['TOTAL'] += 50
            
    elif message == "JOKER" and variabileGlobale['JOKER'] == "":
        if variabileGlobale['TOTAL'] == "":
            variabileGlobale['TOTAL'] = 0
        if variabileGlobale['JOKER'] == "":
            variabileGlobale['JOKER'] = 0
        for i in zaruri:
            variabileGlobale['JOKER'] += i
            variabileGlobale['TOTAL'] += i


start()

Serverul asculta pe adresa:  ('127.0.0.1', 8881)
Accesat de catre:  ('127.0.0.1', 53664)
Accesat de catre:  ('127.0.0.1', 53665)
